# Домашнее задание к лекции "Понятие класса"
Преподаватель: **Константин Башевой**

09.04.2020

## Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

## Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [1]:
from libs.exchange import Rate

In [49]:
class RateWithMaxAndDiff(Rate):
    def __init__(self):
        super().__init__(format ='full')
    
    def currency_max(self):  # ЗАДАНИЕ 1
        """ Получение названия валюты с максимальным значением курса """
        r = self.exchange_rates()

        # формирование списка пар <название валюты>-<текущий курс>
        currency_list = [(r[currency]['Name'], r[currency]['Value']) for currency in r]       
                                 
        # выбор пары с максимальным курсом       
        max_ = max(currency_list, key = lambda x: x[1])  
#         currency_list.sort(key = lambda x: x[1])  # сортивровка списка по значению курса                    
        
        return max_[0]

In [50]:
cur = RateWithMaxAndDiff()
print(f'Валюта с максимальным значением курса: {cur.currency_max()}')

Валюта с максимальным значением курса: СДР (специальные права заимствования)
